In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import mplhep as hep
hep.style.use("CMS")
from coffea import util
import itertools
import os, sys
import glob
import copy
import uproot
import json

import mplhep as hep
hep.style.use("CMS")

sys.path.append('../python/')
import functions

In [3]:
IOV = '2018'

In [4]:
saved = '../local/history/medium/coffea/scale'


qcd16 = util.load(f'{saved}/QCD_2016all.coffea')
qcd17 = util.load(f'{saved}/QCD_2017.coffea')
qcd18 = util.load(f'{saved}/QCD_2018.coffea')


ttbar17 = util.load(f'{saved}/TTbar_2017.coffea')
ttbar16 = util.load(f'{saved}/TTbar_2016all.coffea')
ttbar18 = util.load(f'{saved}/TTbar_2018.coffea')


data17 = util.load(f'{saved}/JetHT_2017.coffea')
data16 = util.load(f'{saved}/JetHT_2016all.coffea')
data18 = util.load(f'{saved}/JetHT_2018.coffea')


# qcd17 = util.load(f'../outputs/scale/QCD_2017.coffea')
# ttbar17 = util.load(f'../outputs/scale/TTbar_2017.coffea')
# data17 = util.load(f'../outputs/scale/JetHT_2017.coffea')

# qcd16 = util.load(f'../outputs/scale/QCD_2016all.coffea')
# ttbar16 = util.load(f'../outputs/scale/TTbar_2016all.coffea')
# data16 = util.load(f'../outputs/scale/JetHT_2016all.coffea')

## Cutflow

In [5]:
data = {}
cuts = []

for cut in data16['cutflow'].keys():
    if 'sumw' in cut: continue
        
    evts = data18['cutflow'][cut] + data17['cutflow'][cut] + data16['cutflow'][cut]
    data[cut] = evts
    cuts.append(evts)

        

In [6]:
df = pd.DataFrame(data=cuts, index=data.keys())
df

,0
all events,253897484
trigger,16032275
htCut,6455322
metfilter,6415349
jetid,6415276
jetkincut,6354100
twoFatJets,5644143
oneTTbar,5644143
dPhiCut,5584764
Good Subjets,5444719


In [7]:
table1 = '''
\\begin{table}
    \\begin{center}
        \\rowcolors{2}{gray!15}{white}
        \\begin{tabular}{ |p{6cm}||p{3cm}|  } \\hline 
            \\textbf{Cut} & \\textbf{Events} \\\\ \\hline
            All Events                                  &  '''+'{0:0,.0f}'.format(data['all events'])+''' \\\\
            Trigger                                     &  '''+'{0:0,.0f}'.format(data['trigger'])+''' \\\\ 
            $HT > 1400$                                 &   '''+'{0:0,.0f}'.format(data['htCut'])+''' \\\\ 
            METfilter                                   &   '''+'{0:0,.0f}'.format(data['metfilter'])+''' \\\\ 
            $\\ge 1$ AK8 jet passing JetID               &   '''+'{0:0,.0f}'.format(data['jetid'])+''' \\\\ 
            $\\ge 1$ AK8 $p_T > 400$ GeV, $|y| < 2.4$    &   '''+'{0:0,.0f}'.format(data['jetkincut'])+''' \\\\ 
            $>2$ AK8 Jets                               &   '''+'{0:0,.0f}'.format(data['twoFatJets'])+''' \\\\ 
            %$\\geq 1$ ttbar candidate                   &   '''+'{0:0,.0f}'.format(data['oneTTbar'])+''' \\\\
            $|\\Delta\\varphi| > 2.1$                     &   '''+'{0:0,.0f}'.format(data['dPhiCut'])+''' \\\\ 
            \\hline
        \\end{tabular}
        \\caption{Cutflow for blinded JetHT events for 2016, 2017 and 2018 datasets.}
        \\label{tab:cutflow}
    \\end{center}
\\end{table}

'''

In [8]:
print(table1)


\begin{table}
    \begin{center}
        \rowcolors{2}{gray!15}{white}
        \begin{tabular}{ |p{6cm}||p{3cm}|  } \hline 
            \textbf{Cut} & \textbf{Events} \\ \hline
            All Events                                  &  253,897,484 \\
            Trigger                                     &  16,032,275 \\ 
            $HT > 1400$                                 &   6,455,322 \\ 
            METfilter                                   &   6,415,349 \\ 
            $\ge 1$ AK8 jet passing JetID               &   6,415,276 \\ 
            $\ge 1$ AK8 $p_T > 400$ GeV, $|y| < 2.4$    &   6,354,100 \\ 
            $>2$ AK8 Jets                               &   5,644,143 \\ 
            %$\geq 1$ ttbar candidate                   &   5,644,143 \\
            $|\Delta\varphi| > 2.1$                     &   5,584,764 \\ 
            \hline
        \end{tabular}
        \caption{Cutflow for blinded JetHT events for 2016, 2017 and 2018 datasets.}
        \label{tab:cutflow}
   

## Weights

In [9]:
wgts = {}
wgts['2018'] = ttbar18['weights']
wgts['2017'] = ttbar17['weights']
wgts['2016'] = ttbar16['weights']

wdata = {}
wdata2 = {}

for year in ['2016', '2017', '2018']:
    
    wdata[year] = {}
    wdata2[year] = {}
    nom = wgts[year]['nominal']
    
    for wgt in wgts[year]:
        
#         print(wgt)
        
        
        if not 'nominal' in wgt:
            
            try:

                wdata2[year][wgt] = wgts[year][wgt]/nom

                wdata[year][wgt] = np.abs(1-wgts[year][wgt]/nom) #np.abs(1-np.abs(wgts[wgt]/nom))
            except:
                continue


In [22]:
for year in wdata2.keys():
    print(year)
    
    for key in wdata2[year].keys():
    
        print(key, '{0:0.3f}'.format(wdata2[year][key]), '{0:0.3f}'.format(wdata[year][key]))

2016
pdfDown 1.017 0.017
q2Down 0.700 0.300
btagDown 1.000 0.000
prefiringUp 0.990 0.010
btagUp 1.000 0.000
prefiringDown 1.010 0.010
pileupDown 0.997 0.003
pileupUp 1.003 0.003
q2Up 1.468 0.468
pdfUp 1.009 0.009
jesUp 1.050 0.050
jesDown 0.571 0.429
jerUp 0.401 0.599
jerDown 0.199 0.801
2017
btagDown 1.000 0.000
pdfUp 1.009 0.009
btagUp 1.000 0.000
q2Up 1.466 0.466
q2Down 0.701 0.299
pileupDown 0.994 0.006
prefiringDown 1.010 0.010
prefiringUp 0.990 0.010
pdfDown 1.017 0.017
pileupUp 1.006 0.006
jesUp 1.049 0.049
jesDown 0.572 0.428
jerUp 0.402 0.598
jerDown 0.199 0.801
2018
pdfDown 1.183 0.183
q2Down 0.814 0.186
btagDown 1.162 0.162
btagUp 1.162 0.162
pileupUp 1.170 0.170
pdfUp 1.172 0.172
q2Up 1.705 0.705
pileupDown 1.155 0.155
jesUp 1.060 0.060
jesDown 0.629 0.371
jerUp 0.502 0.498
jerDown 0.332 0.668
hem 0.162 0.838


In [17]:
table2 = '''
\\begin{table}
    \\begin{center}
        \\begin{tabular}{ |p{1cm}||p{3cm}| |p{3cm}| } \\hline 
            \\textbf{Year} & \\textbf{Systematic} & \\textbf{up/down} \\\\ \\hline
              2016   & PileUp       &   '''+'+{0:0.2f}/-{1:0.2f}'.format(wdata['2016']['pileupUp'], wdata['2016']['pileupDown'])+''' \\\\
                     & PDF          &   '''+'+{0:0.2f}/-{1:0.2f}'.format(wdata['2016']['pdfUp'],    wdata['2016']['pdfDown'])+''' \\\\ 
                     & $Q^2$        &   '''+'+{0:0.2f}/-{1:0.2f}'.format(wdata['2016']['q2Up'],     wdata['2016']['q2Down'])+''' \\\\ 
                     & btag         &   '''+'+{0:0.2f}/-{1:0.2f}'.format(wdata['2016']['btagUp'],   wdata['2016']['btagDown'])+''' \\\\ 
                     & JES          &   '''+'-{0:0.2f}/-{1:0.2f}'.format(wdata['2016']['jesUp'],    wdata['2016']['jesDown'])+''' \\\\ 
                     & JER          &   '''+'-{0:0.2f}/-{1:0.2f}'.format(wdata['2016']['jerUp'],    wdata['2016']['jerDown'])+''' \\\\
                     & L1 Prefiring &   '''+'+{0:0.2f}/+{1:0.2f}'.format(wdata['2017']['prefiringUp'],    wdata['2017']['prefiringDown'])+''' \\\\
              \\hline    
              2017   & PileUp       &   '''+'+{0:0.2f}/-{1:0.2f}'.format(wdata['2017']['pileupUp'], wdata['2017']['pileupDown'])+''' \\\\
                     & PDF          &   '''+'+{0:0.2f}/-{1:0.2f}'.format(wdata['2017']['pdfUp'],    wdata['2017']['pdfDown'])+''' \\\\ 
                     & $Q^2$        &   '''+'+{0:0.2f}/-{1:0.2f}'.format(wdata['2017']['q2Up'],     wdata['2017']['q2Down'])+''' \\\\ 
                     & btag         &   '''+'+{0:0.2f}/-{1:0.2f}'.format(wdata['2017']['btagUp'],   wdata['2017']['btagDown'])+''' \\\\ 
                     & JES          &   '''+'-{0:0.2f}/-{1:0.2f}'.format(wdata['2017']['jesUp'],    wdata['2017']['jesDown'])+''' \\\\ 
                     & JER          &   '''+'-{0:0.2f}/-{1:0.2f}'.format(wdata['2017']['jerUp'],    wdata['2017']['jerDown'])+''' \\\\
                     & L1 Prefiring &   '''+'+{0:0.2f}/+{1:0.2f}'.format(wdata['2017']['prefiringUp'],    wdata['2017']['prefiringDown'])+''' \\\\
              \\hline
              2018   & PileUp       &   '''+'+{0:0.2f}/-{1:0.2f}'.format(wdata['2018']['pileupUp'], wdata['2018']['pileupDown'])+''' \\\\
                     & PDF          &   '''+'+{0:0.2f}/-{1:0.2f}'.format(wdata['2018']['pdfUp'],    wdata['2018']['pdfDown'])+''' \\\\ 
                     & $Q^2$        &   '''+'+{0:0.2f}/-{1:0.2f}'.format(wdata['2018']['q2Up'],     wdata['2018']['q2Down'])+''' \\\\ 
                     & btag         &   '''+'+{0:0.2f}/-{1:0.2f}'.format(wdata['2018']['btagUp'],   wdata['2018']['btagDown'])+''' \\\\ 
                     & JES          &   '''+'-{0:0.2f}/-{1:0.2f}'.format(wdata['2018']['jesUp'],    wdata['2018']['jesDown'])+''' \\\\ 
                     & JER          &   '''+'-{0:0.2f}/-{1:0.2f}'.format(wdata['2018']['jerUp'],    wdata['2018']['jerDown'])+''' \\\\
                     & HEM          &   '''+'-{0:0.2f}'.format(wdata['2018']['hem'])+''' \\\\

               \\hline
        \\end{tabular}
        \\label{tab:syst}
        \\caption{Systematic uncertainties for 2016, 2017, 2018 MC TTbar datasets}
    \\end{center}
\\end{table}

'''

In [18]:
print(table2)


\begin{table}
    \begin{center}
        \begin{tabular}{ |p{1cm}||p{3cm}| |p{3cm}| } \hline 
            \textbf{Year} & \textbf{Systematic} & \textbf{up/down} \\ \hline
              2016   & PileUp       &   +0.00/-0.00 \\
                     & PDF          &   +0.01/-0.02 \\ 
                     & $Q^2$        &   +0.47/-0.30 \\ 
                     & btag         &   +0.00/-0.00 \\ 
                     & JES          &   -0.05/-0.43 \\ 
                     & JER          &   -0.60/-0.80 \\
                     & L1 Prefiring &   +0.01/+0.01 \\
              \hline    
              2017   & PileUp       &   +0.01/-0.01 \\
                     & PDF          &   +0.01/-0.02 \\ 
                     & $Q^2$        &   +0.47/-0.30 \\ 
                     & btag         &   +0.00/-0.00 \\ 
                     & JES          &   -0.05/-0.43 \\ 
                     & JER          &   -0.60/-0.80 \\
                     & L1 Prefiring &   +0.01/+0.01 \\
              \hline
    

In [ ]:
'+{0:0.2f}/-{1:0.2f}'.format(wdata['pileupUp'],wdata['pileupDown'])

## Event count comparisons

### raw events

In [519]:
# scenario1 = cmsv2 + csvv2
# scenario2 = cmsv2 + deepcsv
# scenario3 = deepak8 + deepcsv


# s1_file1 = util.load('../outputs/TTbar_2016_700to1000_cmstop_csvv2.coffea')
# s1_file2 = util.load('../outputs/TTbar_2016_1000toInf_cmstop_csvv2.coffea')
# s1_file3 = util.load('../outputs/TTbar_2016APV_700to1000_cmstop_csvv2.coffea')
# s1_file4 = util.load('../outputs/TTbar_2016APV_1000toInf_cmstop_csvv2.coffea')

s1_file1 = util.load('../outputs/TTbar_2016_700to1000_cmstop_csvv2_HT950.coffea')
s1_file2 = util.load('../outputs/TTbar_2016_1000toInf_cmstop_csvv2_HT950.coffea')
s1_file3 = util.load('../outputs/TTbar_2016APV_700to1000_cmstop_csvv2_HT950.coffea')
s1_file4 = util.load('../outputs/TTbar_2016APV_1000toInf_cmstop_csvv2_HT950.coffea')

s2_file1 = util.load('../outputs/TTbar_2016_700to1000_cmstop_deepcsv_HT950.coffea')
s2_file2 = util.load('../outputs/TTbar_2016_1000toInf_cmstop_deepcsv_HT950.coffea')
s2_file3 = util.load('../outputs/TTbar_2016APV_700to1000_cmstop_deepcsv_HT950.coffea')
s2_file4 = util.load('../outputs/TTbar_2016APV_1000toInf_cmstop_deepcsv_HT950.coffea')

s3_file1 = util.load('../outputs/TTbar_2016_700to1000_HT950.coffea')
s3_file2 = util.load('../outputs/TTbar_2016_1000toInf_HT950.coffea')
s3_file3 = util.load('../outputs/TTbar_2016APV_700to1000_HT950.coffea')
s3_file4 = util.load('../outputs/TTbar_2016APV_1000toInf_HT950.coffea')



# s1_file1 = util.load('../outputs/table/TTbar_2016_700to1000_cmstop_csvv2_HT950_test.coffea')
# s1_file2 = util.load('../outputs/table/TTbar_2016_1000toInf_cmstop_csvv2_HT950_test.coffea')
# s1_file3 = util.load('../outputs/table/TTbar_2016APV_700to1000_cmstop_csvv2_HT950_test.coffea')
# s1_file4 = util.load('../outputs/table/TTbar_2016APV_1000toInf_cmstop_csvv2_HT950_test.coffea')

# s2_file1 = util.load('../outputs/table/TTbar_2016_700to1000_cmstop_deepcsv_HT950_test.coffea')
# s2_file2 = util.load('../outputs/table/TTbar_2016_1000toInf_cmstop_deepcsv_HT950_test.coffea')
# s2_file3 = util.load('../outputs/table/TTbar_2016APV_700to1000_cmstop_deepcsv_HT950_test.coffea')
# s2_file4 = util.load('../outputs/table/TTbar_2016APV_1000toInf_cmstop_deepcsv_HT950_test.coffea')

# s3_file1 = util.load('../outputs/TTbar_2016_700to1000_HT950_test.coffea')
# s3_file2 = util.load('../outputs/TTbar_2016_1000toInf_HT950_test.coffea')
# s3_file3 = util.load('../outputs/TTbar_2016APV_700to1000_HT950_test.coffea')
# s3_file4 = util.load('../outputs/TTbar_2016APV_1000toInf_HT950_test.coffea')


    
label_map = s3_file1['analysisCategories']
label_to_int = {label: i for i, label in label_map.items()}
label_to_int
    
    
    


{'at0bcen': 0,
 'at0bfwd': 1,
 'at1bcen': 2,
 'at1bfwd': 3,
 'at2bcen': 4,
 'at2bfwd': 5,
 'pret0bcen': 6,
 'pret0bfwd': 7,
 'pret1bcen': 8,
 'pret1bfwd': 9,
 'pret2bcen': 10,
 'pret2bfwd': 11,
 '2t0bcen': 12,
 '2t0bfwd': 13,
 '2t1bcen': 14,
 '2t1bfwd': 15,
 '2t2bcen': 16,
 '2t2bfwd': 17}

### scaled events

In [520]:
functions.xs['TTbar']['700to1000'], functions.xs['TTbar']['1000toInf']

(76.605096, 20.5777424)

In [521]:
s3_file1['cutflow']

defaultdict_accumulator(int,
                        {'all events 1': 33502717,
                         'all events': 33502717,
                         'sumw': 22590033619.9375,
                         'sumw2': 15815461933440.0,
                         'htCut': 1719787,
                         'jetid': 1719111,
                         'jetkincut': 1094695,
                         'twoFatJets': 371408,
                         'oneTTbar': 371408,
                         'dPhiCut': 362396,
                         'Good Subjets': 356783,
                         'at0bcen': 17930,
                         'at0bfwd': 10335,
                         'at1bcen': 32496,
                         'at1bfwd': 13932,
                         'at2bcen': 13646,
                         'at2bfwd': 2511,
                         'pret0bcen': 12837,
                         'pret0bfwd': 7877,
                         'pret1bcen': 48263,
                         'pret1bfwd': 23841,
              

## $N_{events}$

In [522]:
ttbar_2016_700to1000_events    = 33502717
ttbar_2016_1000toInf_events    = 23673116
ttbar_2016APV_700to1000_events = 23727154
ttbar_2016APV_1000toInf_events = 23208356



print(s1_file1['cutflow']['all events'] / ttbar_2016_700to1000_events)
print(s1_file2['cutflow']['all events'] / ttbar_2016_1000toInf_events)
print(s1_file3['cutflow']['all events'] / ttbar_2016APV_700to1000_events)
print(s1_file4['cutflow']['all events'] / ttbar_2016APV_1000toInf_events)

print(s2_file1['cutflow']['all events'] / ttbar_2016_700to1000_events)
print(s2_file2['cutflow']['all events'] / ttbar_2016_1000toInf_events)
print(s2_file3['cutflow']['all events'] / ttbar_2016APV_700to1000_events)
print(s2_file4['cutflow']['all events'] / ttbar_2016APV_1000toInf_events)

print(s3_file1['cutflow']['all events'] / ttbar_2016_700to1000_events)
print(s3_file2['cutflow']['all events'] / ttbar_2016_1000toInf_events)
print(s3_file3['cutflow']['all events'] / ttbar_2016APV_700to1000_events)
print(s3_file4['cutflow']['all events'] / ttbar_2016APV_1000toInf_events)

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


### scale factors

In [523]:
scaling = True

s1f1 = 1.0
s1f2 = 1.0
s1f3 = 1.0
s1f4 = 1.0

s2f1 = 1.0
s2f2 = 1.0
s2f3 = 1.0
s2f4 = 1.0

s3f1 = 1.0
s3f2 = 1.0
s3f3 = 1.0
s3f4 = 1.0

factor = 16

if scaling:
    
    s1f1 = functions.lumi['2016'] * functions.xs['TTbar']['700to1000'] / s1_file1['cutflow']['all events']
    s1f2 = functions.lumi['2016'] * functions.xs['TTbar']['1000toInf'] / s1_file2['cutflow']['all events']
    s1f3 = functions.lumi['2016APV'] * functions.xs['TTbar']['700to1000'] / s1_file3['cutflow']['all events']
    s1f4 = functions.lumi['2016APV'] * functions.xs['TTbar']['1000toInf'] / s1_file4['cutflow']['all events']

    s2f1 = functions.lumi['2016'] * functions.xs['TTbar']['700to1000'] / s2_file1['cutflow']['all events']
    s2f2 = functions.lumi['2016'] * functions.xs['TTbar']['1000toInf'] / s2_file2['cutflow']['all events']
    s2f3 = functions.lumi['2016APV'] * functions.xs['TTbar']['700to1000'] / s2_file3['cutflow']['all events']
    s2f4 = functions.lumi['2016APV'] * functions.xs['TTbar']['1000toInf'] / s2_file4['cutflow']['all events']
   
    s3f1 = functions.lumi['2016'] * functions.xs['TTbar']['700to1000'] / s3_file1['cutflow']['all events']
    s3f2 = functions.lumi['2016'] * functions.xs['TTbar']['1000toInf'] / s3_file2['cutflow']['all events']
    s3f3 = functions.lumi['2016APV'] * functions.xs['TTbar']['700to1000'] / s3_file3['cutflow']['all events']
    s3f4 = functions.lumi['2016APV'] * functions.xs['TTbar']['1000toInf'] / s3_file4['cutflow']['all events']

    
#     s1f1 = functions.lumi['2016'] * functions.xs['TTbar']['700to1000'] / s1_file1['cutflow']['sumw']
#     s1f2 = functions.lumi['2016'] * functions.xs['TTbar']['1000toInf'] / s1_file2['cutflow']['sumw']
#     s1f3 = functions.lumi['2016APV'] * functions.xs['TTbar']['700to1000'] / s1_file3['cutflow']['sumw']
#     s1f4 = functions.lumi['2016APV'] * functions.xs['TTbar']['1000toInf'] / s1_file4['cutflow']['sumw']

#     s2f1 = functions.lumi['2016'] * functions.xs['TTbar']['700to1000'] / s2_file1['cutflow']['sumw']
#     s2f2 = functions.lumi['2016'] * functions.xs['TTbar']['1000toInf'] / s2_file2['cutflow']['sumw']
#     s2f3 = functions.lumi['2016APV'] * functions.xs['TTbar']['700to1000'] / s2_file3['cutflow']['sumw']
#     s2f4 = functions.lumi['2016APV'] * functions.xs['TTbar']['1000toInf'] / s2_file4['cutflow']['sumw']
    

#     s3f1 = functions.lumi['2016'] * functions.xs['TTbar']['700to1000'] / s3_file1['cutflow']['sumw']
#     s3f2 = functions.lumi['2016'] * functions.xs['TTbar']['1000toInf'] / s3_file2['cutflow']['sumw']
#     s3f3 = functions.lumi['2016APV'] * functions.xs['TTbar']['700to1000'] / s3_file3['cutflow']['sumw']
#     s3f4 = functions.lumi['2016APV'] * functions.xs['TTbar']['1000toInf'] / s3_file4['cutflow']['sumw']

    
    
    
#     s1f1 = functions.lumi['2016'] * functions.xs['TTbar']['700to1000']    / ttbar_2016_700to1000_events   
#     s1f2 = functions.lumi['2016'] * functions.xs['TTbar']['1000toInf']    / ttbar_2016_1000toInf_events   
#     s1f3 = functions.lumi['2016APV'] * functions.xs['TTbar']['700to1000'] / ttbar_2016APV_700to1000_events
#     s1f4 = functions.lumi['2016APV'] * functions.xs['TTbar']['1000toInf'] / ttbar_2016APV_1000toInf_events

#     s2f1 = functions.lumi['2016'] * functions.xs['TTbar']['700to1000']    / ttbar_2016_700to1000_events   
#     s2f2 = functions.lumi['2016'] * functions.xs['TTbar']['1000toInf']    / ttbar_2016_1000toInf_events   
#     s2f3 = functions.lumi['2016APV'] * functions.xs['TTbar']['700to1000'] / ttbar_2016APV_700to1000_events
#     s2f4 = functions.lumi['2016APV'] * functions.xs['TTbar']['1000toInf'] / ttbar_2016APV_1000toInf_events
   
#     s3f1 = functions.lumi['2016'] * functions.xs['TTbar']['700to1000']    / ttbar_2016_700to1000_events   
#     s3f2 = functions.lumi['2016'] * functions.xs['TTbar']['1000toInf']    / ttbar_2016_1000toInf_events   
#     s3f3 = functions.lumi['2016APV'] * functions.xs['TTbar']['700to1000'] / ttbar_2016APV_700to1000_events
#     s3f4 = functions.lumi['2016APV'] * functions.xs['TTbar']['1000toInf'] / ttbar_2016APV_1000toInf_events





In [524]:
s3_file4['cutflow']

defaultdict_accumulator(int,
                        {'all events 1': 23208356,
                         'all events': 23208356,
                         'sumw': 15118461280.875,
                         'sumw2': 10955853821712.0,
                         'htCut': 4324485,
                         'jetid': 4323536,
                         'jetkincut': 3733294,
                         'twoFatJets': 2349569,
                         'oneTTbar': 2349569,
                         'dPhiCut': 2326290,
                         'Good Subjets': 2289142,
                         'at0bcen': 84590,
                         'at0bfwd': 67603,
                         'at1bcen': 185057,
                         'at1bfwd': 141657,
                         'at2bcen': 102432,
                         'at2bfwd': 73209,
                         'pret0bcen': 81535,
                         'pret0bfwd': 59909,
                         'pret1bcen': 324256,
                         'pret1bfwd': 231097,
    

In [525]:



s1_events_0bcen = np.sum([
    s1f1*np.sum(s1_file1['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bcen']}].values()),
    s1f2*np.sum(s1_file2['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bcen']}].values()),
    s1f3*np.sum(s1_file3['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bcen']}].values()),
    s1f4*np.sum(s1_file4['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bcen']}].values())
])


s1_events_1bcen = np.sum([
    s1f1*np.sum(s1_file1['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bcen']}].values()),
    s1f2*np.sum(s1_file2['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bcen']}].values()),
    s1f3*np.sum(s1_file3['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bcen']}].values()),
    s1f4*np.sum(s1_file4['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bcen']}].values())
])


s1_events_2bcen = np.sum([
    s1f1*np.sum(s1_file1['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bcen']}].values()),
    s1f2*np.sum(s1_file2['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bcen']}].values()),
    s1f3*np.sum(s1_file3['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bcen']}].values()),
    s1f4*np.sum(s1_file4['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bcen']}].values())
])


s1_events_0bfwd = np.sum([
    s1f1*np.sum(s1_file1['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bfwd']}].values()),
    s1f2*np.sum(s1_file2['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bfwd']}].values()),
    s1f3*np.sum(s1_file3['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bfwd']}].values()),
    s1f4*np.sum(s1_file4['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bfwd']}].values())
])


s1_events_1bfwd = np.sum([
    s1f1*np.sum(s1_file1['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bfwd']}].values()),
    s1f2*np.sum(s1_file2['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bfwd']}].values()),
    s1f3*np.sum(s1_file3['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bfwd']}].values()),
    s1f4*np.sum(s1_file4['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bfwd']}].values())
])


s1_events_2bfwd = np.sum([
    s1f1*np.sum(s1_file1['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bfwd']}].values()),
    s1f2*np.sum(s1_file2['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bfwd']}].values()),
    s1f3*np.sum(s1_file3['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bfwd']}].values()),
    s1f4*np.sum(s1_file4['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bfwd']}].values())
])






s2_events_0bcen = np.sum([
    s2f1*np.sum(s2_file1['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bcen']}].values()),
    s2f2*np.sum(s2_file2['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bcen']}].values()),
    s2f3*np.sum(s2_file3['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bcen']}].values()),
    s2f4*np.sum(s2_file4['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bcen']}].values())
])


s2_events_1bcen = np.sum([
    s2f1*np.sum(s2_file1['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bcen']}].values()),
    s2f2*np.sum(s2_file2['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bcen']}].values()),
    s2f3*np.sum(s2_file3['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bcen']}].values()),
    s2f4*np.sum(s2_file4['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bcen']}].values())
])


s2_events_2bcen = np.sum([
    s2f1*np.sum(s2_file1['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bcen']}].values()),
    s2f2*np.sum(s2_file2['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bcen']}].values()),
    s2f3*np.sum(s2_file3['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bcen']}].values()),
    s2f4*np.sum(s2_file4['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bcen']}].values())
])


s2_events_0bfwd = np.sum([
    s2f1*np.sum(s2_file1['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bfwd']}].values()),
    s2f2*np.sum(s2_file2['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bfwd']}].values()),
    s2f3*np.sum(s2_file3['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bfwd']}].values()),
    s2f4*np.sum(s2_file4['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bfwd']}].values())
])


s2_events_1bfwd = np.sum([
    s2f1*np.sum(s2_file1['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bfwd']}].values()),
    s2f2*np.sum(s2_file2['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bfwd']}].values()),
    s2f3*np.sum(s2_file3['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bfwd']}].values()),
    s2f4*np.sum(s2_file4['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bfwd']}].values())
])


s2_events_2bfwd = np.sum([
    s2f1*np.sum(s2_file1['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bfwd']}].values()),
    s2f2*np.sum(s2_file2['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bfwd']}].values()),
    s2f3*np.sum(s2_file3['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bfwd']}].values()),
    s2f4*np.sum(s2_file4['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bfwd']}].values())
])








s3_events_0bcen = np.sum([
    s3f1*np.sum(s3_file1['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bcen']}].values()),
    s3f2*np.sum(s3_file2['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bcen']}].values()),
    s3f3*np.sum(s3_file3['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bcen']}].values()),
    s3f4*np.sum(s3_file4['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bcen']}].values())
])


s3_events_1bcen = np.sum([
    s3f1*np.sum(s3_file1['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bcen']}].values()),
    s3f2*np.sum(s3_file2['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bcen']}].values()),
    s3f3*np.sum(s3_file3['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bcen']}].values()),
    s3f4*np.sum(s3_file4['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bcen']}].values())
])


s3_events_2bcen = np.sum([
    s3f1*np.sum(s3_file1['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bcen']}].values()),
    s3f2*np.sum(s3_file2['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bcen']}].values()),
    s3f3*np.sum(s3_file3['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bcen']}].values()),
    s3f4*np.sum(s3_file4['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bcen']}].values())
])


s3_events_0bfwd = np.sum([
    s3f1*np.sum(s3_file1['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bfwd']}].values()),
    s3f2*np.sum(s3_file2['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bfwd']}].values()),
    s3f3*np.sum(s3_file3['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bfwd']}].values()),
    s3f4*np.sum(s3_file4['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t0bfwd']}].values())
])


s3_events_1bfwd = np.sum([
    s3f1*np.sum(s3_file1['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bfwd']}].values()),
    s3f2*np.sum(s3_file2['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bfwd']}].values()),
    s3f3*np.sum(s3_file3['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bfwd']}].values()),
    s3f4*np.sum(s3_file4['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t1bfwd']}].values())
])


s3_events_2bfwd = np.sum([
    s3f1*np.sum(s3_file1['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bfwd']}].values()),
    s3f2*np.sum(s3_file2['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bfwd']}].values()),
    s3f3*np.sum(s3_file3['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bfwd']}].values()),
    s3f4*np.sum(s3_file4['ttbarmass'][{'systematic':'nominal', 'anacat':label_to_int['2t2bfwd']}].values())
])


s1_events = s1_events_0bcen + s1_events_1bcen + s1_events_2bcen + s1_events_0bfwd + s1_events_1bfwd + s1_events_2bfwd
s2_events = s2_events_0bcen + s2_events_1bcen + s2_events_2bcen + s2_events_0bfwd + s2_events_1bfwd + s2_events_2bfwd
s3_events = s3_events_0bcen + s3_events_1bcen + s3_events_2bcen + s3_events_0bfwd + s3_events_1bfwd + s3_events_2bfwd





In [526]:

s1_events_0bcen = np.sum([
    s1f1*s1_file1['cutflow']['2t0bcen'],
    s1f2*s1_file2['cutflow']['2t0bcen'],
    s1f3*s1_file3['cutflow']['2t0bcen'],
    s1f4*s1_file4['cutflow']['2t0bcen']
])


s1_events_1bcen = np.sum([
    s1f1*s1_file1['cutflow']['2t1bcen'],
    s1f2*s1_file2['cutflow']['2t1bcen'],
    s1f3*s1_file3['cutflow']['2t1bcen'],
    s1f4*s1_file4['cutflow']['2t1bcen']
])

s1_events_2bcen = np.sum([
    s1f1*s1_file1['cutflow']['2t2bcen'],
    s1f2*s1_file2['cutflow']['2t2bcen'],
    s1f3*s1_file3['cutflow']['2t2bcen'],
    s1f4*s1_file4['cutflow']['2t2bcen']
])


s1_events_0bfwd = np.sum([
    s1f1*s1_file1['cutflow']['2t0bfwd'],
    s1f2*s1_file2['cutflow']['2t0bfwd'],
    s1f3*s1_file3['cutflow']['2t0bfwd'],
    s1f4*s1_file4['cutflow']['2t0bfwd']
])


s1_events_1bfwd = np.sum([
    s1f1*s1_file1['cutflow']['2t1bfwd'],
    s1f2*s1_file2['cutflow']['2t1bfwd'],
    s1f3*s1_file3['cutflow']['2t1bfwd'],
    s1f4*s1_file4['cutflow']['2t1bfwd']
])

s1_events_2bfwd = np.sum([
    s1f1*s1_file1['cutflow']['2t2bfwd'],
    s1f2*s1_file2['cutflow']['2t2bfwd'],
    s1f3*s1_file3['cutflow']['2t2bfwd'],
    s1f4*s1_file4['cutflow']['2t2bfwd']
])


s1_events_cutflow = np.sum([
    s1f1*s1_file1['cutflow']['Good Subjets'],
    s1f2*s1_file2['cutflow']['Good Subjets'],
    s1f3*s1_file3['cutflow']['Good Subjets'],
    s1f4*s1_file4['cutflow']['Good Subjets']
])


s2_events_0bcen = np.sum([
    s2f1*s2_file1['cutflow']['2t0bcen'],
    s2f2*s2_file2['cutflow']['2t0bcen'],
    s2f3*s2_file3['cutflow']['2t0bcen'],
    s2f4*s2_file4['cutflow']['2t0bcen']
])


s2_events_1bcen = np.sum([
    s2f1*s2_file1['cutflow']['2t1bcen'],
    s2f2*s2_file2['cutflow']['2t1bcen'],
    s2f3*s2_file3['cutflow']['2t1bcen'],
    s2f4*s2_file4['cutflow']['2t1bcen']
])

s2_events_2bcen = np.sum([
    s2f1*s2_file1['cutflow']['2t2bcen'],
    s2f2*s2_file2['cutflow']['2t2bcen'],
    s2f3*s2_file3['cutflow']['2t2bcen'],
    s2f4*s2_file4['cutflow']['2t2bcen']
])


s2_events_0bfwd = np.sum([
    s2f1*s2_file1['cutflow']['2t0bfwd'],
    s2f2*s2_file2['cutflow']['2t0bfwd'],
    s2f3*s2_file3['cutflow']['2t0bfwd'],
    s2f4*s2_file4['cutflow']['2t0bfwd']
])


s2_events_1bfwd = np.sum([
    s2f1*s2_file1['cutflow']['2t1bfwd'],
    s2f2*s2_file2['cutflow']['2t1bfwd'],
    s2f3*s2_file3['cutflow']['2t1bfwd'],
    s2f4*s2_file4['cutflow']['2t1bfwd']
])

s2_events_2bfwd = np.sum([
    s2f1*s2_file1['cutflow']['2t2bfwd'],
    s2f2*s2_file2['cutflow']['2t2bfwd'],
    s2f3*s2_file3['cutflow']['2t2bfwd'],
    s2f4*s2_file4['cutflow']['2t2bfwd']
])


s2_events_cutflow = np.sum([
    s2f1*s2_file1['cutflow']['Good Subjets'],
    s2f2*s2_file2['cutflow']['Good Subjets'],
    s2f3*s2_file3['cutflow']['Good Subjets'],
    s2f4*s2_file4['cutflow']['Good Subjets']
])



s3_events_0bcen = np.sum([
    s3f1*s3_file1['cutflow']['2t0bcen'],
    s3f2*s3_file2['cutflow']['2t0bcen'],
    s3f3*s3_file3['cutflow']['2t0bcen'],
    s3f4*s3_file4['cutflow']['2t0bcen']
])


s3_events_1bcen = np.sum([
    s3f1*s3_file1['cutflow']['2t1bcen'],
    s3f2*s3_file2['cutflow']['2t1bcen'],
    s3f3*s3_file3['cutflow']['2t1bcen'],
    s3f4*s3_file4['cutflow']['2t1bcen']
])

s3_events_2bcen = np.sum([
    s3f1*s3_file1['cutflow']['2t2bcen'],
    s3f2*s3_file2['cutflow']['2t2bcen'],
    s3f3*s3_file3['cutflow']['2t2bcen'],
    s3f4*s3_file4['cutflow']['2t2bcen']
])


s3_events_0bfwd = np.sum([
    s3f1*s3_file1['cutflow']['2t0bfwd'],
    s3f2*s3_file2['cutflow']['2t0bfwd'],
    s3f3*s3_file3['cutflow']['2t0bfwd'],
    s3f4*s3_file4['cutflow']['2t0bfwd']
])


s3_events_1bfwd = np.sum([
    s3f1*s3_file1['cutflow']['2t1bfwd'],
    s3f2*s3_file2['cutflow']['2t1bfwd'],
    s3f3*s3_file3['cutflow']['2t1bfwd'],
    s3f4*s3_file4['cutflow']['2t1bfwd']
])

s3_events_2bfwd = np.sum([
    s3f1*s3_file1['cutflow']['2t2bfwd'],
    s3f2*s3_file2['cutflow']['2t2bfwd'],
    s3f3*s3_file3['cutflow']['2t2bfwd'],
    s3f4*s3_file4['cutflow']['2t2bfwd']
])


s3_events_cutflow = np.sum([
    s3f1*s3_file1['cutflow']['Good Subjets'],
    s3f2*s3_file2['cutflow']['Good Subjets'],
    s3f3*s3_file3['cutflow']['Good Subjets'],
    s3f4*s3_file4['cutflow']['Good Subjets']
])




s1_events = s1_events_0bcen + s1_events_1bcen + s1_events_2bcen + s1_events_0bfwd + s1_events_1bfwd + s1_events_2bfwd
s2_events = s2_events_0bcen + s2_events_1bcen + s2_events_2bcen + s2_events_0bfwd + s2_events_1bfwd + s2_events_2bfwd
s3_events = s3_events_0bcen + s3_events_1bcen + s3_events_2bcen + s3_events_0bfwd + s3_events_1bfwd + s3_events_2bfwd


In [527]:
print(s3_events, s3_events_cutflow)
print(s3_events_cutflow / s3_events)

21911.484198649214 102237.06429653845
4.665912330249226


In [528]:
string = '''

\\begin{table}
    \\resizebox{0.8\\textwidth}{!}{%
        \\begin{tabular}{@{}cccccc@{}}
        & \\textbf{Category} & \\textbf{2016 AN} & \\textbf{CMSv2 CSVv2} & \\textbf{CMSv2 DeepCSV} & \\textbf{DeepAK8 DeepCSV} \\\\ \\cline{1-6}
        \\multirow{6}{*}{Obs. Data}
        & 0bcen & $      22313$ & $      22447$ & $      20084$ & $      20661$ \\\\ 
        & 0bfwd & $      21836$ & $      21767$ & $      19588$ & $      23187$ \\\\ \\cline{2-6}
        & 1bcen & $\\ \\    9368$ & $\\ \\    8936$ & $      10195$ & $      20581$ \\\\ 
        & 1bfwd & $\\ \\    7973$ & $\\ \\    7667$ & $\\ \\    9178$ & $      20220$ \\\\ \\cline{2-6}
        & 2bcen & $\\ \\    2226$ & $\\ \\    2197$ & $\\ \\    3301$ & $\\ \\    6338$ \\\\ 
        & 2bfwd & $\\ \\    1376$ & $\\ \\    1390$ & $\\ \\    2058$ & $\\ \\    5090$ \\\\ \\cline{1-6}\\\\\\cline{1-6}
        \\multirow{6}{*}{Tot. Bkg.}
        & 0bcen & $      22530$ & $      21258$ & $      19596$ & $      16370$ \\\\ 
        & 0bfwd & $      21388$ & $      20353$ & $      19270$ & $      18680$ \\\\ \\cline{2-6}
        & 1bcen & $\\ \\    9883$ & $\\ \\    8660$ & $      10286$ & $      22285$ \\\\ 
        & 1bfwd & $\\ \\    8227$ & $\\ \\    7268$ & $\\ \\    9148$ & $      22284$ \\\\ \\cline{2-6}
        & 2bcen & $\\ \\    2646$ & $\\ \\    2438$ & $\\ \\    3051$ & $\\ \\    4699$ \\\\ 
        & 2bfwd & $\\ \\    1674$ & $\\ \\    1515$ & $\\ \\    2020$ & $\\ \\    3995$ \\\\ \\cline{1-6}\\\\\\cline{1-6}
        \\multirow{7}{*}{SM $t\\bar{t}$}
        & 0bcen & $\\ \\  ''' + '  1261$ & $\\ \\    {a:0.0f}$ & $\\ \\    {b:0.0f}$ & $\\ \\    {c:0.0f}$ \\\\ '.format(a=s1_events_0bcen, b=s2_events_0bcen, c=s3_events_0bcen) + '''
        & 0bfwd & $\\ \\  ''' + '   812$ & $\\ \\    {a:0.0f}$ & $\\ \\    {b:0.0f}$ & $\\ \\    {c:0.0f}$ \\\\ '.format(a=s1_events_0bfwd, b=s2_events_0bfwd, c=s3_events_0bfwd) + '''
        & 1bcen & $\\ \\  ''' + '  3347$ & $\\ \\    {a:0.0f}$ & $\\ \\    {b:0.0f}$ & $\\ \\    {c:0.0f}$ \\\\ '.format(a=s1_events_1bcen, b=s2_events_1bcen, c=s3_events_1bcen) + '''
        & 1bfwd & $\\ \\  ''' + '  1980$ & $\\ \\    {a:0.0f}$ & $\\ \\    {b:0.0f}$ & $\\ \\    {c:0.0f}$ \\\\ '.format(a=s1_events_1bfwd, b=s2_events_1bfwd, c=s3_events_1bfwd) + '''
        & 2bcen & $\\ \\  ''' + '  2165$ & $\\ \\    {a:0.0f}$ & $\\ \\    {b:0.0f}$ & $\\ \\    {c:0.0f}$ \\\\ '.format(a=s1_events_2bcen, b=s2_events_2bcen, c=s3_events_2bcen) + '''
        & 2bfwd & $\\ \\  ''' + '  1205$ & $\\ \\    {a:0.0f}$ & $\\ \\    {b:0.0f}$ & $\\ \\    {c:0.0f}$ \\\\ '.format(a=s1_events_2bfwd, b=s2_events_2bfwd, c=s3_events_2bfwd) + '''
        & Total & $\\ \\  ''' + '  10770$ & $\\ \\    {a:0.0f}$ & $\\ \\    {b:0.0f}$ & $\\ \\    {c:0.0f}$ \\\\ '.format(a=s1_events, b=s2_events, c=s3_events) + '''
        
      \\end{tabular}
      \\caption{Comparison of event counts between the older analysis ({\\bf 2016 AN}) and this analysis, using the old taggers
        ({\\bf CMSv2 \\& CSVv2}) and the newest criteria used in this analysis ({\\bf DeepAK8 \\& DeepCSV}).
        {\\it Note: Our background estimate and $t\\bar{t}$ event counts were taken pre-fit. The NTMJ component of the total background was estimated by using mistag rates, rather than "pass/fail functions".Also, we apply a NLO correction to $t\\bar{t}$ (which was not present in the 2016 analysis) when using one of the latest taggers.}}
      \\label{Table1_2016}
\\end{table}


'''
print(string)






\begin{table}
    \resizebox{0.8\textwidth}{!}{%
        \begin{tabular}{@{}cccccc@{}}
        & \textbf{Category} & \textbf{2016 AN} & \textbf{CMSv2 CSVv2} & \textbf{CMSv2 DeepCSV} & \textbf{DeepAK8 DeepCSV} \\ \cline{1-6}
        \multirow{6}{*}{Obs. Data}
        & 0bcen & $      22313$ & $      22447$ & $      20084$ & $      20661$ \\ 
        & 0bfwd & $      21836$ & $      21767$ & $      19588$ & $      23187$ \\ \cline{2-6}
        & 1bcen & $\ \    9368$ & $\ \    8936$ & $      10195$ & $      20581$ \\ 
        & 1bfwd & $\ \    7973$ & $\ \    7667$ & $\ \    9178$ & $      20220$ \\ \cline{2-6}
        & 2bcen & $\ \    2226$ & $\ \    2197$ & $\ \    3301$ & $\ \    6338$ \\ 
        & 2bfwd & $\ \    1376$ & $\ \    1390$ & $\ \    2058$ & $\ \    5090$ \\ \cline{1-6}\\\cline{1-6}
        \multirow{6}{*}{Tot. Bkg.}
        & 0bcen & $      22530$ & $      21258$ & $      19596$ & $      16370$ \\ 
        & 0bfwd & $      21388$ & $      20353$ & $      19270$ & $   

## Parameters

In [7]:
params_2016 = json.load(open('../data/corrections/params_2016.json'))
params_2017 = json.load(open('../data/corrections/params_2017.json'))
params_2018 = json.load(open('../data/corrections/params_2018.json'))


In [8]:
params_2017['0bcen']

{'param': [0.0177125482904, 1.26850375461],
 'error': [0.00421965280724, 0.0280109421751],
 'function': '0x1'}

In [9]:
param_table = '''
\\begin{table}[htbp]
    \\centering
        \\label{tab:syst}
       \\caption{Values for parameters of the 2016 transfer function}
        \\begin{tabular}{|c||c||c||c||c|} \\hline 
            \\textbf{Year} & \\textbf{Region} & \\textbf{Parameter} & \\textbf{Value} & \\textbf{Uncertainty} \\\\ \\hline
            \\multirow{20}{*}{2016}
              
                     & \\multirow{2}{*}{cen0b}  '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['0bcen']['param'][0], params_2016['0bcen']['error'][0])+'''
                     '''+'&       & p1 &  {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['0bcen']['param'][1], params_2016['0bcen']['error'][1])+''' \\cline{2-5}
                     
                     & \\multirow{4}{*}{cen1b}  '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['1bcen']['param'][0], params_2016['1bcen']['error'][0])+''' 
                     '''+'&       & p1 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['1bcen']['param'][1], params_2016['1bcen']['error'][1])+''' 
                     '''+'&       & p2 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['1bcen']['param'][2], params_2016['1bcen']['error'][2])+'''
                     '''+'&       & p3 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['1bcen']['param'][3], params_2016['1bcen']['error'][3])+''' \\cline{2-5}
                     
                     & \\multirow{4}{*}{cen2b}  '''+'& p0 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['2bcen']['param'][0], params_2016['2bcen']['error'][0])+''' 
                     '''+'&       & p1 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['2bcen']['param'][1], params_2016['2bcen']['error'][0])+''' 
                     '''+'&       & p2 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['2bcen']['param'][2], params_2016['2bcen']['error'][0])+''' 
                     '''+'&       & p3 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['2bcen']['param'][3], params_2016['2bcen']['error'][0])+'''  \\cline{2-5}
                     
                     & \\multirow{2}{*}{fwd0b} '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['0bfwd']['param'][0], params_2016['0bfwd']['error'][0])+'''
                     '''+'&       & p1 &  {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['0bfwd']['param'][1], params_2016['0bfwd']['error'][1])+''' \\cline{2-5}
                     
                     & \\multirow{4}{*}{fwd1b} '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['1bfwd']['param'][0], params_2016['1bfwd']['error'][0])+''' 
                     '''+'&       & p1 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['1bfwd']['param'][1], params_2016['1bfwd']['error'][1])+''' 
                     '''+'&       & p2 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['1bfwd']['param'][2], params_2016['1bfwd']['error'][2])+'''
                     '''+'&       & p3 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['1bfwd']['param'][3], params_2016['1bfwd']['error'][3])+'''  \\cline{2-5}
                     
                     & \\multirow{4}{*}{fwd2b} '''+'& p0 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['2bfwd']['param'][0], params_2016['2bfwd']['error'][0])+''' 
                     '''+'&       & p1 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['2bfwd']['param'][1], params_2016['2bfwd']['error'][1])+''' 
                     '''+'&       & p2 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['2bfwd']['param'][2], params_2016['2bfwd']['error'][2])+''' 
                     '''+'&       & p3 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['2bfwd']['param'][3], params_2016['2bfwd']['error'][3])+'''  
         
        \\hline
        \\end{tabular}
\\end{table}

\\begin{table}[htbp]
    \\centering
        \\label{tab:syst}
       \\caption{Values for parameters of the 2017 transfer function}
        \\begin{tabular}{|c||c||c||c||c|} \\hline 
            \\textbf{Year} & \\textbf{Region} & \\textbf{Parameter} & \\textbf{Value} & \\textbf{Uncertainty} \\\\ \\hline
            \\multirow{20}{*}{2017}
              
                     & \\multirow{2}{*}{cen0b}  '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['0bcen']['param'][0], params_2017['0bcen']['error'][0])+'''
                     '''+'&       & p1 &  {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['0bcen']['param'][1], params_2017['0bcen']['error'][1])+''' \\cline{2-5}
                     
                     & \\multirow{4}{*}{cen1b}  '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['1bcen']['param'][0], params_2017['1bcen']['error'][0])+''' 
                     '''+'&       & p1 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['1bcen']['param'][1], params_2017['1bcen']['error'][1])+''' 
                     '''+'&       & p2 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['1bcen']['param'][2], params_2017['1bcen']['error'][2])+'''
                     '''+'&       & p3 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['1bcen']['param'][3], params_2017['1bcen']['error'][3])+''' \\cline{2-5}
                     
                     & \\multirow{4}{*}{cen2b}  '''+'& p0 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['2bcen']['param'][0], params_2017['2bcen']['error'][0])+''' 
                     '''+'&       & p1 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['2bcen']['param'][1], params_2017['2bcen']['error'][0])+''' 
                     '''+'&       & p2 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['2bcen']['param'][2], params_2017['2bcen']['error'][0])+''' 
                     '''+'&       & p3 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['2bcen']['param'][3], params_2017['2bcen']['error'][0])+'''  \\cline{2-5}
                     
                     & \\multirow{2}{*}{fwd0b} '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['0bfwd']['param'][0], params_2017['0bfwd']['error'][0])+'''
                     '''+'&       & p1 &  {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['0bfwd']['param'][1], params_2017['0bfwd']['error'][1])+''' \\cline{2-5}
                     
                     & \\multirow{4}{*}{fwd1b} '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['1bfwd']['param'][0], params_2017['1bfwd']['error'][0])+''' 
                     '''+'&       & p1 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['1bfwd']['param'][1], params_2017['1bfwd']['error'][1])+''' 
                     '''+'&       & p2 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['1bfwd']['param'][2], params_2017['1bfwd']['error'][2])+'''
                     '''+'&       & p3 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['1bfwd']['param'][3], params_2017['1bfwd']['error'][3])+'''  \\cline{2-5}
                     
                     & \\multirow{4}{*}{fwd2b} '''+'& p0 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['2bfwd']['param'][0], params_2017['2bfwd']['error'][0])+''' 
                     '''+'&       & p1 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['2bfwd']['param'][1], params_2017['2bfwd']['error'][1])+''' 
                     '''+'&       & p2 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['2bfwd']['param'][2], params_2017['2bfwd']['error'][2])+''' 
                     '''+'&       & p3 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['2bfwd']['param'][3], params_2017['2bfwd']['error'][3])+'''  
         
        \\hline
        \\end{tabular}
\\end{table}


\\begin{table}[htbp]
    \\centering
        \\label{tab:syst}
       \\caption{Values for parameters of the 2018 transfer function}
        \\begin{tabular}{|c||c||c||c||c|} \\hline 
            \\textbf{Year} & \\textbf{Region} & \\textbf{Parameter} & \\textbf{Value} & \\textbf{Uncertainty} \\\\ \\hline
            \\multirow{20}{*}{2018}
              
                     & \\multirow{2}{*}{cen0b}  '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['0bcen']['param'][0], params_2018['0bcen']['error'][0])+'''
                     '''+'&       & p1 &  {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['0bcen']['param'][1], params_2018['0bcen']['error'][1])+''' \\cline{2-5}
                     
                     & \\multirow{4}{*}{cen1b}  '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bcen']['param'][0], params_2018['1bcen']['error'][0])+''' 
                     '''+'&       & p1 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bcen']['param'][1], params_2018['1bcen']['error'][1])+''' 
                     '''+'&       & p2 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bcen']['param'][2], params_2018['1bcen']['error'][2])+'''
                     '''+'&       & p3 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bcen']['param'][3], params_2018['1bcen']['error'][3])+''' \\cline{2-5}
                     
                     & \\multirow{4}{*}{cen2b}  '''+'& p0 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bcen']['param'][0], params_2018['2bcen']['error'][0])+''' 
                     '''+'&       & p1 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bcen']['param'][1], params_2018['2bcen']['error'][0])+''' 
                     '''+'&       & p2 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bcen']['param'][2], params_2018['2bcen']['error'][0])+''' 
                     '''+'&       & p3 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bcen']['param'][3], params_2018['2bcen']['error'][0])+'''  \\cline{2-5}
                     
                     & \\multirow{2}{*}{fwd0b} '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['0bfwd']['param'][0], params_2018['0bfwd']['error'][0])+'''
                     '''+'&       & p1 &  {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['0bfwd']['param'][1], params_2018['0bfwd']['error'][1])+''' \\cline{2-5}
                     
                     & \\multirow{4}{*}{fwd1b} '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bfwd']['param'][0], params_2018['1bfwd']['error'][0])+''' 
                     '''+'&       & p1 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bfwd']['param'][1], params_2018['1bfwd']['error'][1])+''' 
                     '''+'&       & p2 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bfwd']['param'][2], params_2018['1bfwd']['error'][2])+'''
                     '''+'&       & p3 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bfwd']['param'][3], params_2018['1bfwd']['error'][3])+'''  \\cline{2-5}
                     
                     & \\multirow{4}{*}{fwd2b} '''+'& p0 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bfwd']['param'][0], params_2018['2bfwd']['error'][0])+''' 
                     '''+'&       & p1 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bfwd']['param'][1], params_2018['2bfwd']['error'][1])+''' 
                     '''+'&       & p2 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bfwd']['param'][2], params_2018['2bfwd']['error'][2])+''' 
                     '''+'&       & p3 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bfwd']['param'][3], params_2018['2bfwd']['error'][3])+'''  
         
        \\hline
        \\end{tabular}
\\end{table}


'''
print(param_table)


\begin{table}[htbp]
    \centering
        \label{tab:syst}
       \caption{Values for parameters of the 2016 transfer function}
        \begin{tabular}{|c||c||c||c||c|} \hline 
            \textbf{Year} & \textbf{Region} & \textbf{Parameter} & \textbf{Value} & \textbf{Uncertainty} \\ \hline
            \multirow{20}{*}{2016}
              
                     & \multirow{2}{*}{cen0b}  & p0 & 0.02884 & $\pm$ 0.00519 \\
                     &       & p1 &  1.33217 & $\pm$ 0.03539 \\ \cline{2-5}
                     
                     & \multirow{4}{*}{cen1b}  & p0 & 0.18402 & $\pm$ 0.01350 \\ 
                     &       & p1 & 0.34993 & $\pm$ 0.09814 \\ 
                     &       & p2 & -0.83680 & $\pm$ 0.30567 \\
                     &       & p3 & 0.62839 & $\pm$ 0.26032 \\ \cline{2-5}
                     
                     & \multirow{4}{*}{cen2b}  & p0 &0.95882 & $\pm$ 0.09844 \\ 
                     &       & p1 &2.31152 & $\pm$ 0.09844 \\ 
                     &    

In [11]:
\\multirow{20}{*}{2018}
              
                     & \\multirow{2}{*}{cen0b}  '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['0bcen']['param'][0], params_2018['0bcen']['error'][0])+'''
                     '''+'&       & p1 &  {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['0bcen']['param'][1], params_2018['0bcen']['error'][1])+''' \\cline{2-5}
                     
                     & \\multirow{4}{*}{cen1b}  '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bcen']['param'][0], params_2018['1bcen']['error'][0])+''' 
                     '''+'&       & p1 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bcen']['param'][1], params_2018['1bcen']['error'][1])+''' 
                     '''+'&       & p2 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bcen']['param'][2], params_2018['1bcen']['error'][2])+'''
                     '''+'&       & p3 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bcen']['param'][3], params_2018['1bcen']['error'][3])+'''
                     '''+'&       & p4 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bcen']['param'][4], params_2018['1bcen']['error'][4])+''' \\cline{2-5}
                     
                     & \\multirow{4}{*}{cen2b}  '''+'& p0 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bcen']['param'][0], params_2018['2bcen']['error'][0])+''' 
                     '''+'&       & p1 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bcen']['param'][1], params_2018['2bcen']['error'][1])+''' 
                     '''+'&       & p2 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bcen']['param'][2], params_2018['2bcen']['error'][2])+''' 
                     '''+'&       & p3 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bcen']['param'][3], params_2018['2bcen']['error'][3])+''' 
                     '''+'&       & p4 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bcen']['param'][4], params_2018['2bcen']['error'][4])+'''  \\cline{2-5}
                     
                     & \\multirow{2}{*}{fwd0b} '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['0bfwd']['param'][0], params_2018['0bfwd']['error'][0])+'''
                     '''+'&       & p1 &  {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['0bfwd']['param'][1], params_2018['0bfwd']['error'][1])+''' \\cline{2-5}
                     
                     & \\multirow{4}{*}{fwd1b} '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bfwd']['param'][0], params_2018['1bfwd']['error'][0])+''' 
                     '''+'&       & p1 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bfwd']['param'][1], params_2018['1bfwd']['error'][1])+''' 
                     '''+'&       & p2 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bfwd']['param'][2], params_2018['1bfwd']['error'][2])+'''
                     '''+'&       & p3 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bfwd']['param'][3], params_2018['1bfwd']['error'][3])+'''
                     '''+'&       & p4 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bfwd']['param'][4], params_2018['1bfwd']['error'][4])+'''  \\cline{2-5}
                     
                     & \\multirow{4}{*}{fwd2b} '''+'& p0 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bfwd']['param'][0], params_2018['2bfwd']['error'][0])+''' 
                     '''+'&       & p1 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bfwd']['param'][1], params_2018['2bfwd']['error'][1])+''' 
                     '''+'&       & p2 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bfwd']['param'][2], params_2018['2bfwd']['error'][2])+''' 
                     '''+'&       & p3 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bfwd']['param'][3], params_2018['2bfwd']['error'][3])+''' 
                     '''+'&       & p4 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bfwd']['param'][4], params_2018['2bfwd']['error'][4])+'''  
         
        \\hline

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 33)

In [12]:
param_table_for_overleaf = '''
\\begin{frame}{Background Estimate: Transfer Function}
\\begin{table}[htbp]
    \\centering
        \\label{tab:syst}
       \\caption{Values for parameters of the 2016 transfer function}
        \\resizebox{0.5\\textwidth}{!}{%
        \\begin{tabular}{@{}|c||c||c||c||c|@{}}  \\hline 
            \\textbf{Year} & \\textbf{Region} & \\textbf{Parameter} & \\textbf{Value} & \\textbf{Uncertainty} \\\\ \\hline
            \\multirow{20}{*}{2016}
              
                     & \\multirow{2}{*}{cen0b}  '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['0bcen']['param'][0], params_2016['0bcen']['error'][0])+'''
                     '''+'&       & p1 &  {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['0bcen']['param'][1], params_2016['0bcen']['error'][1])+''' \\cline{2-5}
                     
                     & \\multirow{4}{*}{cen1b}  '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['1bcen']['param'][0], params_2016['1bcen']['error'][0])+''' 
                     '''+'&       & p1 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['1bcen']['param'][1], params_2016['1bcen']['error'][1])+''' 
                     '''+'&       & p2 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['1bcen']['param'][2], params_2016['1bcen']['error'][2])+'''
                     '''+'&       & p3 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['1bcen']['param'][3], params_2016['1bcen']['error'][3])+'''
                     '''+'&       & p4 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['1bcen']['param'][4], params_2016['1bcen']['error'][4])+''' \\cline{2-5}
                     
                     & \\multirow{4}{*}{cen2b}  '''+'& p0 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['2bcen']['param'][0], params_2016['2bcen']['error'][0])+''' 
                     '''+'&       & p1 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['2bcen']['param'][1], params_2016['2bcen']['error'][1])+''' 
                     '''+'&       & p2 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['2bcen']['param'][2], params_2016['2bcen']['error'][2])+''' 
                     '''+'&       & p3 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['2bcen']['param'][3], params_2016['2bcen']['error'][3])+''' 
                     '''+'&       & p4 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['2bcen']['param'][4], params_2016['2bcen']['error'][4])+'''  \\cline{2-5}
                     
                     & \\multirow{2}{*}{fwd0b} '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['0bfwd']['param'][0], params_2016['0bfwd']['error'][0])+'''
                     '''+'&       & p1 &  {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['0bfwd']['param'][1], params_2016['0bfwd']['error'][1])+''' \\cline{2-5}
                     
                     & \\multirow{4}{*}{fwd1b} '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['1bfwd']['param'][0], params_2016['1bfwd']['error'][0])+''' 
                     '''+'&       & p1 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['1bfwd']['param'][1], params_2016['1bfwd']['error'][1])+''' 
                     '''+'&       & p2 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['1bfwd']['param'][2], params_2016['1bfwd']['error'][2])+'''
                     '''+'&       & p3 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['1bfwd']['param'][3], params_2016['1bfwd']['error'][3])+'''
                     '''+'&       & p4 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['1bfwd']['param'][4], params_2016['1bfwd']['error'][4])+'''  \\cline{2-5}
                     
                     & \\multirow{4}{*}{fwd2b} '''+'& p0 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['2bfwd']['param'][0], params_2016['2bfwd']['error'][0])+''' 
                     '''+'&       & p1 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['2bfwd']['param'][1], params_2016['2bfwd']['error'][1])+''' 
                     '''+'&       & p2 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['2bfwd']['param'][2], params_2016['2bfwd']['error'][2])+''' 
                     '''+'&       & p3 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['2bfwd']['param'][3], params_2016['2bfwd']['error'][3])+''' 
                     '''+'&       & p4 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2016['2bfwd']['param'][4], params_2016['2bfwd']['error'][4])+'''  
         
        \\hline
        \\end{tabular}
        }
\\end{table}
\\end{frame}


\\begin{frame}{Background Estimate: Transfer Function}
\\begin{table}[htbp]
    \\centering
        \\label{tab:syst}
       \\caption{Values for parameters of the 2017 transfer function}
        \\resizebox{0.5\\textwidth}{!}{%
        \\begin{tabular}{@{}|c||c||c||c||c|@{}}  \\hline 
            \\textbf{Year} & \\textbf{Region} & \\textbf{Parameter} & \\textbf{Value} & \\textbf{Uncertainty} \\\\ \\hline
            \\multirow{20}{*}{2017}
              
                     & \\multirow{2}{*}{cen0b}  '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['0bcen']['param'][0], params_2017['0bcen']['error'][0])+'''
                     '''+'&       & p1 &  {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['0bcen']['param'][1], params_2017['0bcen']['error'][1])+''' \\cline{2-5}
                     
                     & \\multirow{4}{*}{cen1b}  '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['1bcen']['param'][0], params_2017['1bcen']['error'][0])+''' 
                     '''+'&       & p1 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['1bcen']['param'][1], params_2017['1bcen']['error'][1])+''' 
                     '''+'&       & p2 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['1bcen']['param'][2], params_2017['1bcen']['error'][2])+'''
                     '''+'&       & p3 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['1bcen']['param'][3], params_2017['1bcen']['error'][3])+'''
                     '''+'&       & p4 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['1bcen']['param'][4], params_2017['1bcen']['error'][4])+''' \\cline{2-5}
                     
                     & \\multirow{4}{*}{cen2b}  '''+'& p0 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['2bcen']['param'][0], params_2017['2bcen']['error'][0])+''' 
                     '''+'&       & p1 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['2bcen']['param'][1], params_2017['2bcen']['error'][1])+''' 
                     '''+'&       & p2 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['2bcen']['param'][2], params_2017['2bcen']['error'][2])+''' 
                     '''+'&       & p3 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['2bcen']['param'][3], params_2017['2bcen']['error'][3])+''' 
                     '''+'&       & p4 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['2bcen']['param'][4], params_2017['2bcen']['error'][4])+'''  \\cline{2-5}
                     
                     & \\multirow{2}{*}{fwd0b} '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['0bfwd']['param'][0], params_2017['0bfwd']['error'][0])+'''
                     '''+'&       & p1 &  {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['0bfwd']['param'][1], params_2017['0bfwd']['error'][1])+''' \\cline{2-5}
                     
                     & \\multirow{4}{*}{fwd1b} '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['1bfwd']['param'][0], params_2017['1bfwd']['error'][0])+''' 
                     '''+'&       & p1 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['1bfwd']['param'][1], params_2017['1bfwd']['error'][1])+''' 
                     '''+'&       & p2 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['1bfwd']['param'][2], params_2017['1bfwd']['error'][2])+'''
                     '''+'&       & p3 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['1bfwd']['param'][3], params_2017['1bfwd']['error'][3])+'''
                     '''+'&       & p4 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['1bfwd']['param'][4], params_2017['1bfwd']['error'][4])+'''  \\cline{2-5}
                     
                     & \\multirow{4}{*}{fwd2b} '''+'& p0 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['2bfwd']['param'][0], params_2017['2bfwd']['error'][0])+''' 
                     '''+'&       & p1 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['2bfwd']['param'][1], params_2017['2bfwd']['error'][1])+''' 
                     '''+'&       & p2 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['2bfwd']['param'][2], params_2017['2bfwd']['error'][2])+''' 
                     '''+'&       & p3 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['2bfwd']['param'][3], params_2017['2bfwd']['error'][3])+''' 
                     '''+'&       & p4 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2017['2bfwd']['param'][4], params_2017['2bfwd']['error'][4])+'''  
         
        \\hline
        \\end{tabular}
        }
\\end{table}
\\end{frame}

\\begin{frame}{Background Estimate: Transfer Function}
\\begin{table}[htbp]
    \\centering
        \\label{tab:syst}
       \\caption{Values for parameters of the 2018 transfer function}
        \\resizebox{0.5\\textwidth}{!}{%
        \\begin{tabular}{@{}|c||c||c||c||c|@{}}  \\hline 
            \\textbf{Year} & \\textbf{Region} & \\textbf{Parameter} & \\textbf{Value} & \\textbf{Uncertainty} \\\\ \\hline
            \\multirow{20}{*}{2018}
              
                     & \\multirow{2}{*}{cen0b}  '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['0bcen']['param'][0], params_2018['0bcen']['error'][0])+'''
                     '''+'&       & p1 &  {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['0bcen']['param'][1], params_2018['0bcen']['error'][1])+''' \\cline{2-5}
                     
                     & \\multirow{4}{*}{cen1b}  '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bcen']['param'][0], params_2018['1bcen']['error'][0])+''' 
                     '''+'&       & p1 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bcen']['param'][1], params_2018['1bcen']['error'][1])+''' 
                     '''+'&       & p2 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bcen']['param'][2], params_2018['1bcen']['error'][2])+'''
                     '''+'&       & p3 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bcen']['param'][3], params_2018['1bcen']['error'][3])+'''
                     '''+'&       & p4 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bcen']['param'][4], params_2018['1bcen']['error'][4])+''' \\cline{2-5}
                     
                     & \\multirow{4}{*}{cen2b}  '''+'& p0 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bcen']['param'][0], params_2018['2bcen']['error'][0])+''' 
                     '''+'&       & p1 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bcen']['param'][1], params_2018['2bcen']['error'][1])+''' 
                     '''+'&       & p2 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bcen']['param'][2], params_2018['2bcen']['error'][2])+''' 
                     '''+'&       & p3 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bcen']['param'][3], params_2018['2bcen']['error'][3])+''' 
                     '''+'&       & p4 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bcen']['param'][4], params_2018['2bcen']['error'][4])+'''  \\cline{2-5}
                     
                     & \\multirow{2}{*}{fwd0b} '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['0bfwd']['param'][0], params_2018['0bfwd']['error'][0])+'''
                     '''+'&       & p1 &  {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['0bfwd']['param'][1], params_2018['0bfwd']['error'][1])+''' \\cline{2-5}
                     
                     & \\multirow{4}{*}{fwd1b} '''+'& p0 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bfwd']['param'][0], params_2018['1bfwd']['error'][0])+''' 
                     '''+'&       & p1 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bfwd']['param'][1], params_2018['1bfwd']['error'][1])+''' 
                     '''+'&       & p2 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bfwd']['param'][2], params_2018['1bfwd']['error'][2])+'''
                     '''+'&       & p3 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bfwd']['param'][3], params_2018['1bfwd']['error'][3])+'''
                     '''+'&       & p4 & {0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['1bfwd']['param'][4], params_2018['1bfwd']['error'][4])+'''  \\cline{2-5}
                     
                     & \\multirow{4}{*}{fwd2b} '''+'& p0 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bfwd']['param'][0], params_2018['2bfwd']['error'][0])+''' 
                     '''+'&       & p1 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bfwd']['param'][1], params_2018['2bfwd']['error'][1])+''' 
                     '''+'&       & p2 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bfwd']['param'][2], params_2018['2bfwd']['error'][2])+''' 
                     '''+'&       & p3 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bfwd']['param'][3], params_2018['2bfwd']['error'][3])+''' 
                     '''+'&       & p4 &{0:0.5f} & $\\pm$ {1:0.5f} \\\\'.format(params_2018['2bfwd']['param'][4], params_2018['2bfwd']['error'][4])+'''  
         
        \\hline
        \\end{tabular}
        }
\\end{table}
\\end{frame}

'''
print(param_table_for_overleaf)


\begin{frame}{Background Estimate: Transfer Function}
\begin{table}[htbp]
    \centering
        \label{tab:syst}
       \caption{Values for parameters of the 2016 transfer function}
        \resizebox{0.5\textwidth}{!}{%
        \begin{tabular}{@{}|c||c||c||c||c|@{}}  \hline 
            \textbf{Year} & \textbf{Region} & \textbf{Parameter} & \textbf{Value} & \textbf{Uncertainty} \\ \hline
            \multirow{20}{*}{2016}
              
                     & \multirow{2}{*}{cen0b}  & p0 & 0.02884 & $\pm$ 0.00519 \\
                     &       & p1 &  1.33217 & $\pm$ 0.03539 \\ \cline{2-5}
                     
                     & \multirow{4}{*}{cen1b}  & p0 & 0.18402 & $\pm$ 0.01350 \\ 
                     &       & p1 & 0.34993 & $\pm$ 0.09814 \\ 
                     &       & p2 & -0.83680 & $\pm$ 0.30567 \\
                     &       & p3 & 0.62839 & $\pm$ 0.26032 \\
                     &       & p4 & 11.05286 & $\pm$ 1.09681 \\ \cline{2-5}
                     
      